# Step 4: Create models with SD

In [1]:
import pandas as pd 
import os 
import sys 
import pickle
import re
import ast

from pycaret.classification import ClassificationExperiment
from pycaret.containers.models.classification import get_all_model_containers

from sklearn.metrics import (classification_report, 
                             roc_auc_score, 
                             matthews_corrcoef,
                             cohen_kappa_score)
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_validate

# Import help methods
sys.path.append('../src')
from utils import (getExperimentConfig, 
                   getPicklesFromDir, 
                   run_pycaret_setup, 
                   translate_model_name,
                   get_synthetic_filepaths_from_original_data_id,
                   convert_and_clean_dict)

from tuning_grids import Grids

# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

In [ ]:
# read performance data from Step 2
model_performance_df = pd.read_csv(folders['model_perf_filepath'])
# Specify the metrics to sort by for choosing best model
# Choose the target metric when tuning the models
sort_by = config['clf']['tuning_param']['optimize']

run_dataset = config['run_dataset']

for settings in dataset_settings:
        
    if run_dataset is not None and settings['meta']['id'] not in run_dataset:
        continue
        
    settings['setup_param']['fold'] = config['clf']['cv_folds']
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False
    target_label=settings['meta']['target']
    
    # Get experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"
     
    # load original dataset
    cols_dtype=None
    if settings['meta']['cols_dtype'] != None:
        cols_dtyped = settings['meta']['cols_dtype']
        
    original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}", dtype=cols_dtype)
    
    X_test_df = original_data.drop(columns=[settings['meta']['target']], axis=1)
    y_test_df = original_data[settings['meta']['target']]
    
    logg_tags = {
        'Trained on': 'synthetic',
        'Tuned on': 'original',
    }
    
    # Filter the DataFrame based on the Dataset id and sort by specified column
    filtered_df = model_performance_df[model_performance_df["Dataset id"] == settings['meta']['id']]
    

    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

    for sd_filename in synthetic_datasets:
        
        sd_id = os.path.splitext(sd_filename)[0]
        quality = re.findall('Q\d+', sd_id)[0]
        sd_path = folders['sd_dir']+sd_filename
        settings['setup_param']['verbose'] = False
        
        s = run_pycaret_setup(sd_path, settings['setup_param'], meta=settings['meta'])

        # Test the models trained on synthetic with the hyperparameters 
        # recived from tuning on original data, using monte carlo stratified cv
        for _, row in filtered_df.iterrows():
        
            ml_model = row.model
            #get hyperparameters as dict
            hyperparameters = ast.literal_eval(row.Params)
            #buggfix: remove 'priors' from hyperparameters
            hyperparameters.pop('priors', None)
                
            # Add custom tags to the logg, defining dataset type, and Id
            logg_tags = {
                'Dataset id': sd_id,
                'model': ml_model,
                'Quality': quality,
                'Trained on': 'synthetic',
                'Tuned on': 'original',
                'SDG': sd_id.split("_")[0],
            }
            # Check if the setup has already been evaluated
            row_exists = ((model_performance_df['Dataset id'] == logg_tags['Dataset id']) & 
                          (model_performance_df['model'] == logg_tags['model']) &
                          (model_performance_df['Tuned on'] == logg_tags['Tuned on']) &
                          (model_performance_df['Trained on'] == logg_tags['Trained on'])).any()
        
            if not row_exists:
                # create & tune model
                # select model based on wheter using GPU or not

                all_models = get_all_model_containers(s)
                model = all_models[ml_model].class_def()
                
                print(f"Tuned on: 'Original'\n{sd_id}-{translate_model_name(ml_model)}")

                tuned_model = model.set_params(**hyperparameters)
                #train the model on synthetic data
                tuned_model = s.create_model(tuned_model)  
                # get validation results
                val_df = s.pull()
                val_score = {}
                val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
                val_score['val_F1'] = val_df['F1']['Mean']

                sss = StratifiedShuffleSplit(n_splits=config['clf']['mccv_iter'], train_size=settings['setup_param']['train_size'])

                metrics_list = []
                #run monte carlo stratified cross-validation using StratifiedShuffelSplit
                for _, test_index in sss.split(X_test_df, y_test_df):
                    x_test = X_test_df.iloc[test_index]
                    y_test = y_test_df.iloc[test_index]

                    x_test_transformed = s.pipeline.transform(x_test)
                    # Rearrange the column order in the same order as the train data
                    x_test_transformed = x_test_transformed[s.X_train_transformed.columns]

                    y_pred = tuned_model.predict(x_test_transformed)

                    metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
                    test_metrics = {
                        "Accuracy": metrics['accuracy'],
                        "Precision_macro": metrics['macro avg']['precision'],
                        "Recall_macro": metrics['macro avg']['recall'],
                        "F1_macro": metrics['macro avg']['f1-score'],
                        "Precision_weighted": metrics['weighted avg']['precision'],
                        "Recall_weighted": metrics['weighted avg']['recall'],
                        "F1_weighted": metrics['weighted avg']['f1-score'],
                        "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                        "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
                    }
                    metrics_list.append(test_metrics)

                # Convert the list of dictionaries to a DataFrame
                metrics_df = pd.DataFrame(metrics_list)

                # Calculate the average of each column
                average_metrics = metrics_df.mean()

                print("Monte carlo strat-cv mean:")
                print(average_metrics)

                # save results
                performance_row = {**logg_tags, **test_metrics}
                performance_row['Params'] = tuned_model.get_params()
                model_performance_df = model_performance_df.append(performance_row, ignore_index=True)
                ########### End test hyper-param ###########        

        # Start testing all models trained and tuned on synthetic
        for ml_model in config['clf']['ml_models']:
            #start log run
            logg_tags['model'] = ml_model
            logg_tags['Tuned on'] = 'synthetic'
            
            # Check if the setup has already been evaluated
            row_exists = ((model_performance_df['Dataset id'] == logg_tags['Dataset id']) & 
                          (model_performance_df['model'] == logg_tags['model']) &
                          (model_performance_df['Tuned on'] == logg_tags['Tuned on']) &
                          (model_performance_df['Trained on'] == logg_tags['Trained on'])).any()

            if not row_exists:
        
                print(f"Tuned on: 'Synthetic'\n{sd_id}-{translate_model_name(ml_model)}")

                # create & tune model
                all_models = get_all_model_containers(s)
                model = all_models[ml_model].class_def()

                # Get tuning grid
                tune_grid = Grids.get_tuning_grid(ml_model)

                # Is buggy, use default tuning by pycaret
                tuned_model = s.tune_model(model, custom_grid=tune_grid, **config['clf']['tuning_param'])

                # get validation results
                val_df = s.pull()
                val_score = {}
                val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
                val_score['val_F1'] = val_df['F1']['Mean']
                
                sss = StratifiedShuffleSplit(n_splits=config['clf']['mccv_iter'], train_size=settings['setup_param']['train_size'])

                metrics_list = []
                #run monte carlo stratified cross-validation using StratifiedShuffelSplit
                for _, test_index in sss.split(X_test_df, y_test_df):
                    x_test = X_test_df.iloc[test_index]
                    y_test = y_test_df.iloc[test_index]

                    x_test_transformed = s.pipeline.transform(x_test)
                    # Rearrange the column order in the same order as the train data
                    x_test_transformed = x_test_transformed[s.X_train_transformed.columns]

                    y_pred = tuned_model.predict(x_test_transformed)

                    metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
                    test_metrics = {
                        "Accuracy": metrics['accuracy'],
                        "Precision_macro": metrics['macro avg']['precision'],
                        "Recall_macro": metrics['macro avg']['recall'],
                        "F1_macro": metrics['macro avg']['f1-score'],
                        "Precision_weighted": metrics['weighted avg']['precision'],
                        "Recall_weighted": metrics['weighted avg']['recall'],
                        "F1_weighted": metrics['weighted avg']['f1-score'],
                        "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                        "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
                    }
                    metrics_list.append(test_metrics)

                # Convert the list of dictionaries to a DataFrame
                metrics_df = pd.DataFrame(metrics_list)

                # Calculate the average of each column
                average_metrics = metrics_df.mean()

                print(average_metrics)

                # save results and parameters
                performance_row = {**logg_tags, **average_metrics}
                performance_row['Params'] = tuned_model.get_params()
                model_performance_df = model_performance_df.append(performance_row, ignore_index=True)

        # update model performance to csv after each sd_id
        model_performance_df.to_csv(folders['model_perf_filepath'], index=False)

,Description,Value
0,Session id,8709
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(614, 9)"
6,Transformed test set shape,"(154, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,None


Dataset id: SD0Q1_0
Model: KNeighborsClassifier
Hyperparameters : {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 32, 'p': 2, 'weights': 'distance'}


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6290,0.6082,0.8235,0.6222,0.7089,0.2242,0.2414
1,0.6290,0.5952,0.8571,0.6250,0.7229,0.2016,0.2259
2,0.6452,0.6265,0.7714,0.6585,0.7105,0.2595,0.2650
3,0.6774,0.6423,0.9143,0.6531,0.7619,0.3026,0.3467
4,0.6393,0.6117,0.8529,0.6304,0.7250,0.2340,0.2576
5,0.6557,0.6187,0.9412,0.6275,0.7529,0.2539,0.3186
6,0.5738,0.5376,0.8529,0.5800,0.6905,0.0800,0.0971
7,0.6066,0.5822,0.7941,0.6136,0.6923,0.1710,0.1822
8,0.6557,0.6264,0.8824,0.6383,0.7407,0.2659,0.2985


Dataset id: SD0Q1_0
Model: LogisticRegression
Hyperparameters : {'C': [0.0001, 0.00026366508987303583, 0.0006951927961775605, 0.0018329807108324356, 0.004832930238571752, 0.012742749857031334, 0.03359818286283781, 0.08858667904100823, 0.23357214690901212, 0.615848211066026, 1.623776739188721, 4.281332398719396, 11.288378916846883, 29.763514416313132, 78.47599703514607, 206.913808111479, 545.5594781168514, 1438.44988828766, 3792.690190732246, 10000.0], 'penalty': ['l2', None]}


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6290,0.6082,0.8235,0.6222,0.7089,0.2242,0.2414
1,0.6290,0.5952,0.8571,0.6250,0.7229,0.2016,0.2259
2,0.6452,0.6265,0.7714,0.6585,0.7105,0.2595,0.2650
3,0.6774,0.6423,0.9143,0.6531,0.7619,0.3026,0.3467
4,0.6393,0.6117,0.8529,0.6304,0.7250,0.2340,0.2576
5,0.6557,0.6187,0.9412,0.6275,0.7529,0.2539,0.3186
6,0.5738,0.5376,0.8529,0.5800,0.6905,0.0800,0.0971
7,0.6066,0.5822,0.7941,0.6136,0.6923,0.1710,0.1822
8,0.6557,0.6264,0.8824,0.6383,0.7407,0.2659,0.2985


...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


Dataset id: SD0Q1_1
Model: KNeighborsClassifier
Hyperparameters : {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan', 'minkowski'], 'p': [1, 2]}


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:25:23
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2023-04-30 16:25:23,645] Searching the best hyperparameters using 614 samples...
